<a href="https://colab.research.google.com/github/ugarciac/TenCrossValidationMLP/blob/master/CrossValScoreMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# coding: utf-8
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import keras

# fix random seed for reproducibility
seed = np.random.seed(0)
# link TXV: https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/

class MultiLayerPerceptron:
    def __init__(self, in_path: str, path_train: str, hidden_layer: int, classes: int,
                epochs:int, batch:int, learning_rate: float, momentum: float,
                 activation_layer_input:str, activation_layer_output:str, loss:str, metric:str, metric_evaluation:str,
                 test_size: float, shuffletxv: bool, split_folds: int):
        self.path_in = in_path
        self.path_train = path_train
        self.hidden_layer = hidden_layer
        self.classes = classes
        self.epochs = epochs
        self.batch = batch
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.activation_layer_input=activation_layer_input
        self.activation_layer_output=activation_layer_output
        self.loss=loss
        self.metric=metric
        self.metric_evaluation=metric_evaluation
        self.test_size=test_size
        self.shuffletxv=shuffletxv
        self.split_folds = split_folds

        self.__GetData()


    def __GetData(self):
        '''
        This method make the exchange of the original classes of training set with the classes generated by the
        genetic algorithm

        Original training set
        CAG: 0
        NAG: 1
        OAG: 2

        training set generate for the genetic algorithm
        CAG: 0
        NAG: 1
        OAG: 2
        NOISY: 3
        '''
        # Open original training set
        dataH = pd.read_csv(self.path_train)
        datasetH = pd.DataFrame(dataH)

        # data to save csv
        self.data_final = datasetH

        # Remove the instances that belong to class 3(remove noisy)
        # resulting training set
        datafinal = datasetH[datasetH['Clase']!=3]
        
        self.__GetCharacteristics(datafinal)

        return 0

    def __GetCharacteristics(self, datafinal):
        '''
        Gets the characteristics of the instances

        :param datafinal: data set
        :return:
        '''
        # Gets characteristics of resulting training set (generated by genetic algorithm)
        X = np.array(datafinal.iloc[:,1:-2])
        Y = np.array(datafinal.iloc[:,-2], dtype=int).reshape(-1,1)

        self.X_train, self.X_test, y_train, y_test = train_test_split(X, Y, test_size=self.test_size, shuffle=self.shuffletxv)

        # Converts outputs to categorical values
        self.y_train = keras.utils.to_categorical(y_train, num_classes=self.classes)
        self.y_test = keras.utils.to_categorical(y_test, num_classes=self.classes)
        
        return 0

    def Create_Model(self):
        def create_model():
            # Create model
            model = Sequential()
            for i in range(1):
                model.add(Dense(self.hidden_layer, input_dim=self.X_train.shape[1], activation=self.activation_layer_input))
                
            model.add(Dense(self.classes, activation=self.activation_layer_output))

            # Optimizer
            sgd = optimizers.SGD(lr=self.learning_rate, momentum=self.momentum)
            # Compile model
            model.compile(loss=self.loss, 
                          optimizer=sgd, 
                          metrics=[self.metric])
            return model
        return create_model
    
    def EvaluateTrainModel(self):
        cb = TensorBoard()
        create_model = self.Create_Model()

        estimator = KerasClassifier(build_fn=create_model, epochs=self.epochs, batch_size=self.batch, verbose=0)

        # Convert classes to real values
        y_true = [np.argmax(self.y_test[datnum]) for datnum in range(self.y_test.shape[0])]
        y_test = [np.argmax(self.y_test[datnum]) for datnum in range(self.y_test.shape[0])]

        score = cross_val_score(estimator, self.X_test, y_test, cv=self.split_folds, scoring=self.metric_evaluation)
        print('f1_score weighted: ', score.mean())
        print("Accuracy : {:0.2f} (+/- {:0.2f})".format(score.mean(), score.std()))
        #
                
    def SaveData_SaveModel(self):
        self.data_final.to_csv('DATAFINAL.csv')

        # save model
        model_yaml = self.network.to_yaml()
        with open('MODEL.yaml', 'w') as yaml_file:
            yaml_file.write(model_yaml)

        # save weights HDF5
        self.network.save_weights('WEIGHTS.h5')
            
        return 0


    def Write_score(self, out_path):
        '''
        this method writes the value obtained for the evaluation measure f1 score (fitness function of the
        genetic algorithm) reached by the neural network, the file is written in a binary file
        :param path_out:
        :return:
        '''
        bin = IOBinFile()
        score = np.array([self.score], dtype=float)
        bin.WriteBinFloatFileIEEE754(out_path, score)

        return score

Using TensorFlow backend.


In [2]:
IN_PATH = ''
OUT_PATH = ''
PATH_TRAIN = 'iris.csv'
HIDDEN_LAYER = 15
CLASSES = 3
EPOCHS = 300
BATCH = 11999
LEARNING_RATE = 0.1
MOMENTUM = 0.6
ACTIVATION_LAYER_INPUT = 'relu'
ACTIVATION_LAYER_OUTPUT = 'softmax'
LOSS = 'categorical_crossentropy'
METRIC = 'accuracy'
METRIC_EVALUATION = 'f1_weighted'
TEST_SIZE = 0.3
SHUFFLETXV = True
SPLITFOLDS = 10

# Functioning
mlp = MultiLayerPerceptron(in_path=IN_PATH, path_train=PATH_TRAIN, hidden_layer=HIDDEN_LAYER, classes=CLASSES,
                     epochs=EPOCHS, batch=BATCH, learning_rate=LEARNING_RATE, momentum=MOMENTUM,
                     activation_layer_input=ACTIVATION_LAYER_INPUT, activation_layer_output=ACTIVATION_LAYER_OUTPUT, loss=LOSS,
                     metric=METRIC, metric_evaluation=METRIC_EVALUATION,test_size=TEST_SIZE, shuffletxv=SHUFFLETXV, split_folds=SPLITFOLDS)

mlp.EvaluateTrainModel()






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

f1_score weighted:  0.9766666666666668
Accuracy : 0.98 (+/- 0.07)
